In [1]:
#%%
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import warnings

from snowflake.connector import connect

In [2]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine 

In [12]:
Snowflake_conn = create_engine(URL( 
                          user='marko.talledo@mercadofavo.com',
                          authenticator='externalBrowser',
                          account='vta08432.us-east-1',
                          database='FAVODATA',
                          warehouse='FAVODATA',
                          schema='JOURNEY'
                          ))



In [10]:
QUERY_VR_CARTERAS_DICIEMBRE_FINAL = '''
    WITH VR_VENTA_DICIEMBRE as
(
select
cast(a.create_date_time_tz as date) as calendar_date,
cast(date_trunc(month, a.create_date_time_tz) as date) as calendar_month,
--case when extract(day from calendar_date)<=15 then 1 else 2 as quincena,
a.store_url,
a.leader_name,
--b.tier_net_lm,
a.dynamo_customer_id,
a.order_number,
sum(net_value) AS total
--sum(total) over (partition by a.store_url,quincena) as venta_quincenal,
from journey.base as a
--left join FAVODATA.SNP_SANDBOX.AN_ENTREP_TIER_TEST as b
--on a.store_url = b.store_url
where create_date_time is not null and order_status not in ('CANCEL' ,'OPEN') and cancel_date_time_tz is null and calendar_month = '2022-12-01'
group by 1,2,3,4,5,6
order by 1 desc
),

VR_VENTA_DICIEMBRE_CARTERAS AS (
select a.*, b.mentora, b.orientador, b.flag_emprendedor
from VR_VENTA_DICIEMBRE as a
         left join
     FAVODATA.SNP_UNTRUSTED.CARTERAS_DICIEMBRE as b
     on
         a.store_url = b.emprendedor
where b.flag_retencion = 0 ),


VR_CARTERAS_DICIEMBRE_FINAL AS ( -- esta es una de las tablas finales
    select a.mentora,
           a.orientador,
           a.store_url,
           a.leader_name,
           a.dynamo_customer_id,
           b.customer_name,
           sum(a.total)                                                                as total_cliente,
           case when total_cliente >= 100 then 1 else 0 end                            as flag_bono,
           case when total_cliente < 100 then round(100 - total_cliente, 0) else 0 end as falta
    from VR_VENTA_DICIEMBRE_CARTERAS as a
             left join
         journey.customer as b
         on a.dynamo_customer_id = b.dynamo_customer_id
--poner nombre cliente
    group by 1, 2, 3, 4, 5, 6),


VR_CARTERAS_DICIEMBRE_TOTALES as (
select
mentora,
orientador,
store_url,
count(distinct dynamo_customer_id) as clientes,
sum(flag_bono) as clientes_bono,
case when clientes_bono >=5 then 1 else 0 end flag_graduado
from VR_CARTERAS_DICIEMBRE_FINAL
group by 1,2,3),


VR_CARTERAS_DICIEMBRE_TOTALES_2 as ( -- esta es la otra tabla que necesitas actualizar
select
mentora,
a.orientador,
truncate((sum(flag_graduado)/b.cartera),4) as porc_graduados,
b.cartera,
sum(flag_graduado) as total_graduados,
case
when porc_graduados >= 0.25 then 100
when porc_graduados >= 0.20 then 85
when porc_graduados >=0.15 then 70
when porc_graduados >= 0.10 then 60
when porc_graduados >= 0.06 then 50
when porc_graduados >= 0.03 then 40
else 0
end as ganancia_unitaria,
case
when porc_graduados >= 0.25 then total_graduados*100
when porc_graduados >= 0.20 then total_graduados*85
when porc_graduados >= 0.15 then total_graduados*70
when porc_graduados >= 0.10 then total_graduados*60
when porc_graduados >= 0.06 then total_graduados*50
when porc_graduados >= 0.03 then total_graduados*40
else 0
end as ganancia,
case
when porc_graduados >= 0.25 then 'max'
when 0.2 <= porc_graduados and porc_graduados < 0.25 then 0.25
when 0.15 <= porc_graduados and porc_graduados < 0.20 then 0.20
when 0.10 <= porc_graduados and porc_graduados < 0.15 then 0.15
when 0.06 <= porc_graduados and porc_graduados < 0.10 then 0.10
when 0.03 <= porc_graduados and porc_graduados < 0.06 then 0.06
else 0.03
end as porc_meta,
case
when porc_graduados >= 0.25 then 'max'
when 0.2 <= porc_graduados and porc_graduados < 0.25 then ceil(b.cartera*0.25,0)
when 0.15 <= porc_graduados and porc_graduados < 0.20 then ceil(b.cartera*0.20,0)
when 0.10 <= porc_graduados and porc_graduados < 0.15 then ceil(b.cartera*0.15,0)
when 0.06 <= porc_graduados and porc_graduados < 0.10 then ceil(b.cartera*0.10,0)
when 0.03 <= porc_graduados and porc_graduados < 0.06 then ceil(b.cartera*0.06,0)
else ceil(b.cartera*0.03,0)
end as emprendedores_meta
from VR_CARTERAS_DICIEMBRE_TOTALES as a
left join FAVODATA.SNP_UNTRUSTED.CARTERAS_DICIEMBRE_TOTAL as b
on a.orientador = b.orientador
group by 1,2,4)


select * from FAVODATA.SNP_SANDBOX.VR_CARTERAS_DICIEMBRE_FINAL;
    
    
    '''


In [11]:
QUERY_VR_CARTERAS_DICIEMBRE_TOTALES_2 = '''
    WITH VR_VENTA_DICIEMBRE as
(
select
cast(a.create_date_time_tz as date) as calendar_date,
cast(date_trunc(month, a.create_date_time_tz) as date) as calendar_month,
--case when extract(day from calendar_date)<=15 then 1 else 2 as quincena,
a.store_url,
a.leader_name,
--b.tier_net_lm,
a.dynamo_customer_id,
a.order_number,
sum(net_value) AS total
--sum(total) over (partition by a.store_url,quincena) as venta_quincenal,
from journey.base as a
--left join FAVODATA.SNP_SANDBOX.AN_ENTREP_TIER_TEST as b
--on a.store_url = b.store_url
where create_date_time is not null and order_status not in ('CANCEL' ,'OPEN') and cancel_date_time_tz is null and calendar_month = '2022-12-01'
group by 1,2,3,4,5,6
order by 1 desc
),

VR_VENTA_DICIEMBRE_CARTERAS AS (
select a.*, b.mentora, b.orientador, b.flag_emprendedor
from VR_VENTA_DICIEMBRE as a
         left join
     FAVODATA.SNP_UNTRUSTED.CARTERAS_DICIEMBRE as b
     on
         a.store_url = b.emprendedor
where b.flag_retencion = 0 ),


VR_CARTERAS_DICIEMBRE_FINAL AS ( -- esta es una de las tablas finales
    select a.mentora,
           a.orientador,
           a.store_url,
           a.leader_name,
           a.dynamo_customer_id,
           b.customer_name,
           sum(a.total)                                                                as total_cliente,
           case when total_cliente >= 100 then 1 else 0 end                            as flag_bono,
           case when total_cliente < 100 then round(100 - total_cliente, 0) else 0 end as falta
    from VR_VENTA_DICIEMBRE_CARTERAS as a
             left join
         journey.customer as b
         on a.dynamo_customer_id = b.dynamo_customer_id
--poner nombre cliente
    group by 1, 2, 3, 4, 5, 6),

    
VR_CARTERAS_DICIEMBRE_TOTALES as (
select
mentora,
orientador,
store_url,
count(distinct dynamo_customer_id) as clientes,
sum(flag_bono) as clientes_bono,
case when clientes_bono >=5 then 1 else 0 end flag_graduado
from VR_CARTERAS_DICIEMBRE_FINAL
group by 1,2,3),
    

VR_CARTERAS_DICIEMBRE_TOTALES_2 as ( -- esta es la otra tabla que necesitas actualizar
select
mentora,
a.orientador,
truncate((sum(flag_graduado)/b.cartera),4) as porc_graduados,
b.cartera,
sum(flag_graduado) as total_graduados,
case
when porc_graduados >= 0.25 then 100
when porc_graduados >= 0.20 then 85
when porc_graduados >=0.15 then 70
when porc_graduados >= 0.10 then 60
when porc_graduados >= 0.06 then 50
when porc_graduados >= 0.03 then 40
else 0
end as ganancia_unitaria,
case
when porc_graduados >= 0.25 then total_graduados*100
when porc_graduados >= 0.20 then total_graduados*85
when porc_graduados >= 0.15 then total_graduados*70
when porc_graduados >= 0.10 then total_graduados*60
when porc_graduados >= 0.06 then total_graduados*50
when porc_graduados >= 0.03 then total_graduados*40
else 0
end as ganancia,
case
when porc_graduados >= 0.25 then 'max'
when 0.2 <= porc_graduados and porc_graduados < 0.25 then 0.25
when 0.15 <= porc_graduados and porc_graduados < 0.20 then 0.20
when 0.10 <= porc_graduados and porc_graduados < 0.15 then 0.15
when 0.06 <= porc_graduados and porc_graduados < 0.10 then 0.10
when 0.03 <= porc_graduados and porc_graduados < 0.06 then 0.06
else 0.03
end as porc_meta,
case
when porc_graduados >= 0.25 then 'max'
when 0.2 <= porc_graduados and porc_graduados < 0.25 then ceil(b.cartera*0.25,0)
when 0.15 <= porc_graduados and porc_graduados < 0.20 then ceil(b.cartera*0.20,0)
when 0.10 <= porc_graduados and porc_graduados < 0.15 then ceil(b.cartera*0.15,0)
when 0.06 <= porc_graduados and porc_graduados < 0.10 then ceil(b.cartera*0.10,0)
when 0.03 <= porc_graduados and porc_graduados < 0.06 then ceil(b.cartera*0.06,0)
else ceil(b.cartera*0.03,0)
end as emprendedores_meta
from VR_CARTERAS_DICIEMBRE_TOTALES as a
left join FAVODATA.SNP_UNTRUSTED.CARTERAS_DICIEMBRE_TOTAL as b
on a.orientador = b.orientador
group by 1,2,4)

SELECT * FROM VR_CARTERAS_DICIEMBRE_TOTALES_2; 
'''

In [13]:
### Executes SQL query and data into a Pandas DF
df_Carteras_DIC_Totales_2 = pd.read_sql(con = Snowflake_conn,  sql = QUERY_VR_CARTERAS_DICIEMBRE_TOTALES_2)
df_Carteras_DIC_Totales_Final = pd.read_sql(con = Snowflake_conn,  sql = QUERY_VR_CARTERAS_DICIEMBRE_FINAL)


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [25]:
pip install gspread


  Using cached gspread-5.7.2-py3-none-any.whl (40 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Marko\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [26]:
pip install google-api-python-client oauth2client 

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mysql-connector-python 8.0.30 requires protobuf<=3.20.1,>=3.11.0, but you have protobuf 4.21.12 which is incompatible.
You should consider upgrading via the 'c:\Users\Marko\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [31]:
pip install google-auth

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Marko\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [41]:
pip install google.colab 

  Using cached google-colab-1.0.0.tar.gz (72 kB)
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
Using legacy 'setup.py install' for google.colab, since package 'wheel' is not installed.
Using legacy 'setup.py install' for pandas, since package 'wheel' is not installed.
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.2
    Uninstalling pandas-1.4.2:
      Successfully uninstalled pandas-1.4.2
    Running setup.py install for pandas: started
    Running setup.py install for pandas: finished with status 'error'
  Rolling back uninstall of pandas
  Moving to c:\users\marko\appdata\local\programs\python\python310\lib\site-packages\pandas-1.4.2.dist-info\
   from C:\Users\Marko\AppData\Local\Programs\Python\Python310\Lib\site-packages\~andas-1.4.2.dist-info
  Moving to c:\users\marko\appdata\local\programs\python\python310\lib\site-packages\panda

    ERROR: Command errored out with exit status 1:
     command: 'c:\Users\Marko\AppData\Local\Programs\Python\Python310\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Marko\\AppData\\Local\\Temp\\pip-install-t21j_d8f\\pandas_8af744a704a84e008d0d4e7dec5c2638\\setup.py'"'"'; __file__='"'"'C:\\Users\\Marko\\AppData\\Local\\Temp\\pip-install-t21j_d8f\\pandas_8af744a704a84e008d0d4e7dec5c2638\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record 'C:\Users\Marko\AppData\Local\Temp\pip-record-i3fsn85x\install-record.txt' --single-version-externally-managed --compile --install-headers 'c:\Users\Marko\AppData\Local\Programs\Python\Python310\Include\pandas'
         cwd: C:\Users\Marko\AppData\Local\Temp\pip-install-t21

In [40]:
# Librerias de GSheets 
from google.colab import auth
auth.authenticate_user()


ModuleNotFoundError: No module named 'google.colab'